In [ ]:
from gorillatracker.datasets.bristol import BristolDataset
import pandas as pd
from PIL import Image
from torchvision import transforms, models
import torch
import torch.nn as nn
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score, silhouette_score
from torch.utils.data import DataLoader
from gorillatracker.utils.embedding_generator import generate_embeddings
import torch.nn as nn
from torchvision import models

class CustomResNet(nn.Module):
    def __init__(self):
        super(CustomResNet, self).__init__()
        # Load the pretrained ResNet model
        self.resnet = models.resnet50(pretrained=True)
        # Remove the classification head
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-1])

    def forward(self, x):
        # Forward pass through the ResNet model
        x = self.resnet(x)
        # Flatten the output
        x = x.view(x.size(0), -1)
        return x

tf = transforms.Compose([BristolDataset.get_transforms(), transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

bristol = BristolDataset("/workspaces/gorillatracker/data/splits/ground_truth-bristol-cropped_images_face-closedset-mintraincount-3-seed-42-train-0-val-100-test-0", partition="val", transform=tf)




model = CustomResNet()

df = generate_embeddings(model, dataset=bristol, batch_size=32, worker=10)

# with torch.no_grad():
#     for  in bristol:

# # Convert embeddings to a DataFrame
# embedding_df = pd.DataFrame(embeddings)

# # Define the number of clusters
# num_clusters = 5

# # Perform K-Means clustering
# kmeans = KMeans(n_clusters=num_clusters, random_state=42)
# df['cluster'] = kmeans.fit_predict(embedding_df)

# # Compute Davies-Bouldin score
# db_score = davies_bouldin_score(embedding_df, df['cluster'])

# # Compute Silhouette score
# sil_score = silhouette_score(embedding_df, df['cluster'])

# print(f'Davies-Bouldin Score: {db_score}')
# print(f'Silhouette Score: {sil_score}')


In [ ]:
print(df.head())
print(df["embedding"].shape)
print(type(df["embedding"].iloc[0]))
print(df["embedding"].iloc[0].shape)

In [ ]:
from gorillatracker.scripts.visualize_embeddings import EmbeddingProjector
from io import BytesIO
from bokeh.io import show, output_notebook, reset_output
import base64
import numpy as np

output_notebook()
to_pil_image = transforms.ToPILImage()

images = []
for image in df["input"]:
    buffer = BytesIO()
    image.save(buffer, format="JPEG")
    image_byte = base64.b64encode(buffer.getvalue()).decode("utf-8")
    images.append(image_byte)

ep = EmbeddingProjector()
low_dim_embeddings = ep.reduce_dimensions(np.stack(df["embedding"]))
fig = ep.plot_clusters(
    low_dim_embeddings, df["label"], df["label_string"], images, title="Embedding Projector", figsize=(12, 10)
)
show(fig)
reset_output()

In [ ]:
print("Hello world!")